Привет! Меня зовут Исмаил, я буду проверять твой проект :) Можешь обращаться ко мне на «ты». Если тебе комфортно, то и я буду к тебе так обращаться, если нет, то обязательно скажи об этом.

Пожалуйста, не удаляй мои комментарии, которые я буду оставлять в работе. Это особенно поможет, если твои проекты будут отправлены на повторную проверку. При повторных проверках у комментариев будут приписки: "Вторая итерация", "Третья итерация" и т.д. 

Ты тоже можешь реагировать на мои комментарии, но в таком случае постарайся, чтобы твои комментарии отличались от моих: например, выделяй их своим любимым цветом — так у нас не возникнет путаницы :)

Кстати, про цвета! Мои комментарии будут в следующей цветовой гамме: зелеными, желтыми и красными. Например:

<div class="alert-success"> 
<b>Комментарий ревьюера 👍 </b>
    
В случае, если решение на отдельном шаге является полностью правильным. </div>

<div class="alert-warning"> 
<b>Комментарий ревьюера 💡</b> 
    
В случае, если решение может стать еще лучше с некоторыми корректировками </div>
 
<div class="alert-danger"> 
<b>Комментарий ревьюера ❌</b> 
    
В случае, если решение какого-то шага требует значительных изменений. Проект не может быть принят с первого раза, если ревью содержит комментарии, помеченные этим цветом </div>

# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [132]:
import pandas as pd
data = pd.read_csv('/datasets/users_behavior.csv')
data.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [133]:
# Проверка общих значений датасета
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [134]:
# Проверка на наличие дубликатов
data.duplicated().sum()

0

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц.


сalls — количество звонков,

minutes — суммарная длительность звонков в минутах,

messages — количество sms-сообщений,

mb_used — израсходованный интернет-трафик в Мб,

is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

<div class="alert-success"> 
<b>Комментарий ревьюера 👍</b>

Хороший первичный анализ данных

## Разбейте данные на выборки

Разделим исходные данные на обучающую, валидационную и тестовую выборки. Т.к. нельзя сразу разделить на три отдельные выборки, разделим исходной датасет на обучающую (60%) и оставшуюся 40% разделим повторно между валидационной и тестовой выборкой.

In [135]:
from sklearn.model_selection import train_test_split 
data_train, data_ost = train_test_split(data, test_size=0.4, random_state=12345) # Делим на обучающую и остаток
data_test, data_valid = train_test_split(data_ost, test_size=0.5, random_state=12345)  # Делим на тестовую и валидационную

In [136]:
# Проверим размер датасетов
print(data_train.shape, data_test.shape, data_valid.shape)

(1928, 5) (643, 5) (643, 5)


<div class="alert-success"> 
<b>Комментарий ревьюера 👍</b>

Выборки разделены верно, здорово, что проверены их размеры

## Исследуйте модели

Целевым признаком у нас является значения в столбце is_ultra, которое определяет на каком тарифе находится пользователь. Все остальные столбцы являются признаками, которые мы будем давать модели для ее обучения. Данная задача является задачей классификацией, поэтому для оценки модели будем пользоваться метрикой accuary.

### Модель решающего дерева

Деревья принятия решений (DTS) - это непараметрический метод контролируемого обучения, используемый для классификации и регрессии. Цель состоит в том, чтобы создать модель, которая предсказывает значение целевой переменной путем изучения простых правил принятия решений, выведенных из объектов данных. Дерево можно рассматривать как кусочно-постоянное приближение.

In [137]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

features_train = data_train.drop(['is_ultra'], axis=1) # Сохранение признаков модели
target_train = data_train['is_ultra'] # Сохранение целевого признака
features_valid = data_valid.drop(['is_ultra'], axis=1)
target_valid = data_valid['is_ultra']

best_model = None
best_result = 0
best_depth = 0
for depth in range(1,10):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train) 
    result = model.score(features_valid, target_valid) 
    if result > best_result:
        best_model = model # сохранение наилучшей модели
        best_result = result
        best_depth = depth
        
print("Accuracy наилучшей модели на валидационной выборке:", best_result)
print("Max_depth наилучшей модели на валидационной выборке:",best_depth)

Accuracy наилучшей модели на валидационной выборке: 0.7993779160186625
Max_depth наилучшей модели на валидационной выборке: 7



**Min_samples_leaf (от англ. «минимальное количество объектов в листе»). Листья — это нижние узлы с ответами. А гиперпараметр не разрешает создавать лист, в котором слишком мало объектов обучающей выборки. Создадим цикл по количетству объектов в узле и узнаем какой из них дает лучшие показатели модели** 

In [138]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

features_train = data_train.drop(['is_ultra'], axis=1) # Сохранение признаков модели
target_train = data_train['is_ultra'] # Сохранение целевого признака
features_valid = data_valid.drop(['is_ultra'], axis=1)
target_valid = data_valid['is_ultra'] 
best_model = None
best_result = 0
best_leath = 0
for leaf in range(1,10):
    model = DecisionTreeClassifier(random_state=12345, max_depth=7, min_samples_leaf = leaf)
    model.fit(features_train, target_train) 
    result = model.score(features_valid, target_valid) 
    if result > best_result:
        best_model = model # сохранение наилучшей модели
        best_result = result
        best_leaf = leaf
        
print("Accuracy наилучшей модели на валидационной выборке:", best_result)
print("Min_samples_leaf наилучшей модели на валидационной выборке:",best_leaf)

Accuracy наилучшей модели на валидационной выборке: 0.8009331259720062
Min_samples_leaf наилучшей модели на валидационной выборке: 3


**Сriterion='gini' — критерий Джини, который показывает меру сходства двух наборов данных. Обучаясь, дерево в каждом узле (на каждой развилке) из возможных вопросов задаёт наилучший. Сейчас оно выбирает тот вопрос, для которого критерий Джини показывает, что отнесённые к левой ветке данные меньше всего похожи на те, что в правой. Изменим критерий на 'entropy' и узнаем какой из них дает лучший результат** 

In [139]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

features_train = data_train.drop(['is_ultra'], axis=1) # Сохранение признаков модели
target_train = data_train['is_ultra'] # Сохранение целевого признака
features_valid = data_valid.drop(['is_ultra'], axis=1)
target_valid = data_valid['is_ultra']

best_model = None
best_result = 0
criterions = ['gini', 'entropy']
for crit in criterions:
    model = DecisionTreeClassifier(random_state=12345, criterion=crit,  max_depth=7, min_samples_leaf = 3)
    model.fit(features_train, target_train) 
    result = model.score(features_valid, target_valid) 
    if result > best_result:
        best_model = model # сохранение наилучшей модели
        best_result = result
        best_crit = crit
        
print("Accuracy наилучшей модели на валидационной выборке:", best_result)
print("Criterion наилучшей модели на валидационной выборке:",best_crit)

Accuracy наилучшей модели на валидационной выборке: 0.8009331259720062
Criterion наилучшей модели на валидационной выборке: gini


**Вывод:** Метод решающего дерева показал хороший результат на валидационной выборке, мы изменили 3 гиперпараметра для данной модели max_depth, criterion, min_samples_leaf и нам удалость достичь метрику accuracy 0.8009.

<div class="alert-warning"> 
<b>Комментарий ревьюера 💡 </b>

Есть достаточный перебор гиперпараметров, но: зачем опять создавать features_train и другие переменные? Достаточно одного раза. Так же и с библиотеками - достаточно импортировать их единожды

###  Модель "Случайный лес"

Попробуем новый алгоритм классификации — случайный лес (англ. random forest). Алгоритм обучает большое количество независимых друг от друга деревьев, а потом принимает решение на основе голосования. Случайный лес помогает улучшить результат предсказания и избежать переобучения.

In [140]:
from sklearn.ensemble import RandomForestClassifier

features_train = data_train.drop(['is_ultra'], axis=1)
target_train = data_train['is_ultra']
features_valid = data_valid.drop(['is_ultra'], axis=1)
target_valid = data_valid['is_ultra']

best_model = None
best_result = 0
best_est=0
for est in range(1, 11):
    model = RandomForestClassifier(random_state=12345, n_estimators=est) # обучим модель с заданным количеством деревьев через цикл
    model.fit(features_train, target_train) 
    result = model.score(features_valid, target_valid) 
    if result > best_result:
        best_model = model # сохранение наилучшей модели
        best_result = result
        best_est=est
print("Accuracy наилучшей модели на валидационной выборке:", best_result)
print("Best_est наилучшей модели на валидационной выборке:", best_est)

Accuracy наилучшей модели на валидационной выборке: 0.7869362363919129
Best_est наилучшей модели на валидационной выборке: 8


**Вывод:** В данном примере мы иследовали модель 'Случайный лес' изменняя гиперпараметр количества деревьев нам удалось достичь accuracy 0.7869, при этом показатель n_estimators составил 8.

<div class="alert-warning"> 
<b>Комментарий ревьюера 💡 </b>

Аналогично с прошлым комментарием, в целом все здорово, но можно кое-что подправить

### Логистическая регрессия

In [141]:
from sklearn.linear_model import LogisticRegression 

features_train = data_train.drop(['is_ultra'], axis=1)
target_train = data_train['is_ultra']
features_valid = data_valid.drop(['is_ultra'], axis=1)
target_valid = data_valid['is_ultra']

model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)
        
print("Accuracy модели логистической регрессии:", accuracy_score(target_valid, predictions_valid)) 

Accuracy модели логистической регрессии: 0.6842923794712286


**Вывод:** При логистической регрессии accuracy составило 0.6842, при этом измення число итераций не удается добиться болшей точности модели.

<div class="alert-warning"> 
<b>Комментарий ревьюера 💡 </b>

Тут тоже хотелось бы видеть перебор значений параметров

### Обший вывод

Среди всех трех расмотренных моделей наилучший показатель показала модель DecisionTreeClassifier, на втором месте идет RandomForestClassifier и замыкает тройку LogisticRegression. Можно также заметить, что на параметрах по умолчанию модель решающего дерева дает показатели на валидационной выборке лучше, чем остальные. Изменив гиперпараметры модели удалось достичь максимального accuracy 0.8009. Также можно отметить, что скорость модели решающего дерева также обладает хорошим преимуществом.

## Проверьте модель на тестовой выборке

Т.к. наилучший показатель показало решающее дерево, то проверим модель на тестовой выборке и убедимся в том, что модель работает хорошо.


In [142]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

features_train = data_train.drop(['is_ultra'], axis=1) # Сохранение признаков модели
target_train = data_train['is_ultra'] # Сохранение целевого признака
features_test = data_test.drop(['is_ultra'], axis=1)
target_test = data_test['is_ultra']

model = DecisionTreeClassifier(random_state=12345, criterion='gini',  max_depth=7, min_samples_leaf = 3)
model.fit(features_train, target_train) 
result = model.score(features_test, target_test) 
    
print("Accuracy модели на тестовой выборке:", result)

Accuracy модели на тестовой выборке: 0.7791601866251944


**Вывод:** Как мы видим на тестовой выборке решающее дерево показало accuracy 0.7791, что выходит из нижнего предел нашего задания. По этой причине, можно считать модель отвечает нашим требованиям и может использоваться компанией для рекомендации тарифа пользователю на основании его данных

<div class="alert-success"> 
<b>Комментарий ревьюера 👍 </b>

Модель проверена на тесте

## (бонус) Проверьте модели на адекватность

Не совсем понятно смысл данного задания, мы же уже проверяли модель на тестовой выборке. Что необходимо сделать в данном задании ?

<div class="alert-warning"> 
<b>Комментарий ревьюера 💡 </b>

Можно посмотреть на баланс классов: например, если у нас 90 процентов таргета это "1", а остальные 10 - "0", то мы можем тупо выдавать каждый раз цифру 1 и получать метрику 90 %. Любая модель, которая будет выдавать качество ниже этого - плохая/неадекватная

<div class="alert-warning"> 
<b>Комментарий ревьюера 💡 </b>

Хороший проект, тобою проделана большая работа, ты молодец!

Сперва похвалю за наличие везде промежуточных выводов, комментариев. Хороший код, показываешь навыки владения sklearn. Все требуемые пункты в задании в целом выполнены. Оставил пару желтых комментариев, думаю, это сможет улучшить твой проект. В этом проекте большое внимание уделяется подбору гиперпараметров, но это можно автоматизировать! Советую почитать на досуге про GridsearchCV и Optuna, будет полезно
            
Из курсов и дополнительной информации советую Yandex ML Handbook (https://academy.yandex.ru/handbook/ml) и курсы Евгения Соколова на ФКН (Машинное обучение - 1 с математикой https://github.com/esokolov/ml-course-hse и Основы машинного обучения без глубокой математики https://github.com/hse-ds/iad-intro-ds) - там можешь получить дополнительную информацию, более глубокую, возможно. http://wiki.cs.hse.ru/Заглавная_страница - тут можешь глянуть записи семинаров и лекций и другие материалы соответствующих курсов. 
        
Готов принять проект и так, потому что не было критических ошибок. Можешь отправлять проект без доработок, но если хочешь - можешь сделать бонус)
    
Удачи!

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
